In [1]:
'''
Date: 20230712
Aim: background subtraction using images with water
Author: Yike Xie
'''

'\nDate: 20230712\nAim: background subtraction using images with water\nAuthor: Yike Xie\n'

In [2]:
import numpy as np
import pandas as pd
import os
import cv2
import h5py

import matplotlib.pyplot as plt
from skimage import filters
import pickle

# background substraction

In [3]:
def cell_labels(seg_fn):
    # label roughly segmented cells by ilastik according to their size
    print('import segmentated figure')
    seg = cv2.imread(seg_fn, 2)

    print('get labels of each cell')
    from scipy import ndimage as ndi
    seg = ndi.binary_fill_holes(seg - 1)

    labeled, _ = ndi.label(seg)
    return seg, labeled


def crop_ROI(antt, cell, width, height, labeled, pxs):

    print('find ROI...')
    seg = (labeled == pxs[antt])

    i0s = seg.any(axis=1).nonzero()[0]
    i1s = seg.any(axis=0).nonzero()[0]
    i00, i01 = i0s[0], i0s[-1] + 1
    i10, i11 = i1s[0], i1s[-1] + 1

    print('add margins...')
    margin_px1 = int((width - (i01 - i00)) / 2)
    margin_px2 = int((height - (i11 - i10)) / 2)
    margin_px = max(margin_px1, margin_px2)
    
    i00 = max(0, i00 - margin_px)
    i10 = max(0, i10 - margin_px)
    i01 = min(seg.shape[0], i01 + margin_px)
    i11 = min(seg.shape[1], i11 + margin_px)
    
    return i00, i10, i01, i11

In [4]:
# create image array
def img_array(img):
    files = ['wls_325_414',
             'wls_343_414',
             'wls_370_414',
             'wls_343_451',
             'wls_370_451',
             'wls_373_451',
             'wls_343_575',
             'wls_393_575',
             'wls_406_575',
             'wls_441_575',
             'wls_400_594',
             'wls_406_594',
             'wls_431_594',
             'wls_480_594',
             'wls_339_575']
    
    data = np.empty([15] + list(img[files[0]].shape))
    
    for i, file in enumerate(files):
        data[i] = img[file].astype('uint16')
        
    return data # type uint16

# image blurring
from skimage.filters import gaussian
def blur(img, sigma=1, axis=0):
    blurred_img = gaussian(img, sigma=sigma, truncate=3.5, channel_axis=axis, preserve_range=True)
    return blurred_img # type uint16

# background subtraction
def subtract(img, blurred_B):
    subtracted_img = (img - blurred_B)
    return subtracted_img

# image saving
def save_npz(img, brightfield, mask, name, path=False):
    img_dic = {}
    files = ['wls_325_414',
             'wls_343_414',
             'wls_370_414',
             'wls_343_451',
             'wls_370_451',
             'wls_373_451',
             'wls_343_575',
             'wls_393_575',
             'wls_406_575',
             'wls_441_575',
             'wls_400_594',
             'wls_406_594',
             'wls_431_594',
             'wls_480_594',
             'wls_339_575']
    
    for i, file in enumerate(files):
        img_dic[file] = img[i]
        
    img_dic['brightfield'] = brightfield
    img_dic['segmentation'] = mask
    
    
    if path is not False:
        np.savez_compressed(path + name + '.npz', **img_dic) # type uint16

In [9]:
if False:
    # sequenced single cells
    # load cropped single cells
    figure_npz_fdn = '/home/yike/phd/cancer_cells_img_seq/data/202306_imaging/'
    fdn_cell = '/home/yike/phd/cancer_cells_img_seq/figures/batch_202306/cropped_cell_npz/good/'
    ilastik_seg = '/home/yike/phd/cancer_cells_img_seq/figures/batch_202306/fake_RGB_images/picked_figures/'
    fdn_save = '/home/yike/phd/cancer_cells_img_seq/figures/batch_202306/background_substraction/'

    ## load cell annotation information
    df = pd.read_csv('/home/yike/phd/cancer_cells_img_seq/figures/batch_202306/fake_RGB_images/cell_annotation/cell_annotation.tsv', 
              sep='\t', index_col=0)
    sig_df = df[df['Cell_number'] == 1]
    sig_df['Mask'] = sig_df['Batch'].astype(str) + sig_df['Grid']
    ncells = len(df[df['Cell_number'] == 1].index)

if True:
    # dapi bright cells
    # load cropped single cells
    figure_npz_fdn = '/home/yike/phd/cancer_cells_img_seq/data/202306_imaging/'
    fdn_cell = '/home/yike/phd/cancer_cells_img_seq/figures/batch_202306/dapi_cropped_cell_npz/'
    ilastik_seg = '/home/yike/phd/cancer_cells_img_seq/figures/batch_202306/fake_RGB_images/'
    fdn_save = '/home/yike/phd/cancer_cells_img_seq/figures/batch_202306/dapi_background_substraction/'

    ## load cell annotation information
    df = pd.read_csv('/home/yike/phd/cancer_cells_img_seq/figures/batch_202306/merge_dapi_bf/dapi_cell_annotation.tsv', 
              sep='\t')
    df['Mask'] = df['Batch'].astype(str) + df['Grid'] + '_' + df['Cell_annotation'].astype(str)
    df['Cell'] = df['Mask']
    df = df.set_index('Cell')

    blur_images = ['20230620Dish1_H7_1',
                   '20230620Dish1_H7_2',
                   '20230620Dish1_H7_3',
                   '20230622Dish2_J2_3',
                   '20230622Dish2_J2_4',
                   '20230622Dish6_O17_1'
                ]
    df = df.loc[~ df.index.isin(blur_images)]
    
    sig_df = df.copy()
    ncells = sig_df.shape[0]

In [11]:
# background substraction
i = 0
picked_fns = os.listdir(ilastik_seg + 'picked_figures/')
unpicked_fns = os.listdir(ilastik_seg + 'unpicked_figures/')

for batch in [20230620, 20230621, 20230622]:
batch_fdn = os.path.join(figure_npz_fdn, str(batch) + '_imaging')
w_fns = [i for i in os.listdir(batch_fdn) if 'Water' in i]

print('loading water...')
water1 = img_array(np.load(os.path.join(batch_fdn, w_fns[0])))
water2 = img_array(np.load(os.path.join(batch_fdn, w_fns[1])))

for mask in sig_df[sig_df['Batch'] == batch]['Mask'].unique():
    grid = mask.split('_')[0] + '_' + mask.split('_')[1]
    print('loading...')
    _, labeled = cell_labels(os.path.join(
        ilastik_seg + ['unpicked_figures/', 'picked_figures/'][grid + '_Simple Segmentation.tiff' in picked_fns], 
        grid + '_Simple Segmentation.tiff'))
    pxs = np.bincount(labeled.ravel()).argsort()[::-1]

    for c, row in sig_df[sig_df['Mask'] == mask].iterrows():
        i += 1
        print(f'{i + 1}: {c}')

        antt = int(row['Cell_annotation'])

        cell_npz = np.load(os.path.join(fdn_cell, c + '.npz'))
        cell = img_array(cell_npz) # unit16
        _, w, h = cell.shape

        print('crop...')
        i00, i10, i01, i11 = crop_ROI(antt, c, w, h, labeled, pxs)

        ctl1 = water1[:, i00: i01, i10: i11] # unit16
        ctl2 = water2[:, i00: i01, i10: i11] # unit16

        print('blur water...')
        bl_ctl1 = blur(ctl1, sigma=1, axis=0)
        bl_ctl2 = blur(ctl2, sigma=1, axis=0)

        print('background subtraction...')
        sb_cell = subtract(cell, (bl_ctl1 + bl_ctl2) / 2)

        print('blur cell')
        bl_cell = blur(sb_cell)

        print('save as npz')
        brightfield = cell_npz['brightfield']
        mask = cell_npz['new_segmentation']

        save_npz(bl_cell, brightfield, mask, c, path=fdn_save)   

        print('-------------')

# feature extraction

In [12]:
def extract_features(fdn_sub, fdn_seg, fn):
    img = np.load(os.path.join(fdn_sub, fn))
    mask = img['segmentation']
    seg_img = np.load(os.path.join(fdn_seg, fn))
    
    # Area
    area = mask.sum()
    # Horizontal length
    length = mask.any(axis=0).nonzero()[0]
    length = length[-1] - length[0]
    # Vertical width (waist line)
    width = mask.any(axis=1).nonzero()[0]
    width = width[-1] - width[0]
    # Eccentricity
    ecc = length / width

    # Wavelengths
    wls = [(int(wl.split('_')[1]), int(wl.split('_')[2])) for wl in img.files[:-2]]
    
    # Total Intensity
    total_int = [(img[wl] * mask).sum() for wl in img.files[:-2]]
    
    # Average Intensity
    ave_int = [int_i/area for int_i in total_int]
    
    # spectra
    spectra = [(seg_img[wl] * mask).sum() for wl in img.files[:-2]]
    
    feas = {
        'area': area,
        'length': length,
        'width': width,
        'eccentricity': ecc,
        'image': fn.split('/')[-1].split('.')[0],
        'wavelengths': wls,
        'spectra': spectra, # w/o background substraction
        'total_intensity': total_int, # w/ background substraction
        'ave_intensity': ave_int, # w/ background substraction
    }

    return feas

In [13]:
# cells
if False:
    # sequenced single cells
    fdn_sub = '/home/yike/phd/cancer_cells_img_seq/figures/batch_202306/background_substraction/'
    fdn_seg = '/home/yike/phd/cancer_cells_img_seq/figures/batch_202306/cropped_cell_npz/good/'

if True:
    # dapi bright cells
    fdn_sub = '/home/yike/phd/cancer_cells_img_seq/figures/batch_202306/dapi_background_substraction/'
    fdn_seg = '/home/yike/phd/cancer_cells_img_seq/figures/batch_202306/dapi_cropped_cell_npz/'    
    
features = []
for i, fn in enumerate([i for i in os.listdir(fdn_sub) if '.npz' in i]):
    print('{} cell: {}'.format(i+1, fn.split('.')[0]))
    
    feas = extract_features(fdn_sub, fdn_seg, fn)
    features.append(feas)
features_all = pd.DataFrame(features)

# save features as a pickle file
with open(fdn_sub + 'bkg_sub_features.pkl', 'wb') as f:
    pickle.dump(features_all, f)

1 cell: 20230621Dish5_H8_4
2 cell: 20230622Dish5_N14_6
3 cell: 20230622Dish3_N10_9
4 cell: 20230621Dish5_G13_3
5 cell: 20230620Dish4_O12_5
6 cell: 20230622Dish4_H12_1
7 cell: 20230622Dish2_H7_3
8 cell: 20230620Dish4_K13_4
9 cell: 20230622Dish4_M9_6
10 cell: 20230621Dish2_F13_3
11 cell: 20230622Dish4_L13_5
12 cell: 20230622Dish2_H7_6
13 cell: 20230622Dish6_O6_4
14 cell: 20230622Dish6_O6_3
15 cell: 20230622Dish4_L13_7
16 cell: 20230622Dish5_N14_7
17 cell: 20230622Dish2_H7_5
18 cell: 20230620Dish4_O12_1
19 cell: 20230622Dish2_H7_2
20 cell: 20230622Dish2_H7_9
21 cell: 20230620Dish4_O12_4
22 cell: 20230622Dish2_H7_4
23 cell: 20230620Dish4_O12_6
24 cell: 20230620Dish4_K13_7
25 cell: 20230621Dish2_F13_2
26 cell: 20230620Dish4_O12_2
27 cell: 20230620Dish4_K13_5
28 cell: 20230621Dish5_H8_3
29 cell: 20230622Dish5_N14_8
30 cell: 20230621Dish2_F13_1
31 cell: 20230622Dish4_M9_2
32 cell: 20230622Dish2_H7_7
33 cell: 20230620Dish4_O12_3
34 cell: 20230622Dish3_M9_6
35 cell: 20230622Dish2_J2_10
36 cell:

In [16]:
# bright dapi cells

# check the percentage of cells with positive intensities after background subtraction under each channel
df = pd.DataFrame([], index=features_all['image'])
for i, wl in enumerate(features_all.loc[0]['wavelengths']):
    df['{} {}'.format(str(wl[0]), str(wl[1]))] = [fea[i] for fea in features_all['total_intensity']]
    
neg = []
for i in df.index:
    neg.append(sum(1 for n in df.loc[i] if n < 0))
df['total'] = neg

for col in df.columns:
    n = sum(1 for i in df[col] if i < 0)
    print('{} {}'.format(col, str(n * 100/36) + '%'))

325 414 22.22222222222222%
343 414 13.88888888888889%
370 414 13.88888888888889%
343 451 0.0%
370 451 0.0%
373 451 2.7777777777777777%
343 575 0.0%
393 575 25.0%
406 575 33.333333333333336%
441 575 33.333333333333336%
400 594 33.333333333333336%
406 594 33.333333333333336%
431 594 33.333333333333336%
480 594 33.333333333333336%
339 575 33.333333333333336%
total 0.0%


In [88]:
# sequenced single cells

# check the percentage of cells with positive intensities after background subtraction under each channel
df = pd.DataFrame([], index=features_all['image'])
for i, wl in enumerate(features_all.loc[0]['wavelengths']):
    df['{} {}'.format(str(wl[0]), str(wl[1]))] = [fea[i] for fea in features_all['total_intensity']]
    
neg = []
for i in df.index:
    neg.append(sum(1 for n in df.loc[i] if n < 0))
df['total'] = neg

for col in df.columns:
    n = sum(1 for i in df[col] if i < 0)
    print('{} {}'.format(col, str(n * 100/81) + '%'))

325 414 25.925925925925927%
343 414 11.11111111111111%
370 414 12.345679012345679%
343 451 1.2345679012345678%
370 451 2.4691358024691357%
373 451 2.4691358024691357%
343 575 0.0%
393 575 28.395061728395063%
406 575 33.333333333333336%
441 575 33.333333333333336%
400 594 35.80246913580247%
406 594 38.27160493827161%
431 594 38.27160493827161%
480 594 38.27160493827161%
339 575 40.74074074074074%
total 0.0%


# combine two extracted feature files

In [11]:
import scanpy as sc

In [6]:
feas_fn1 = '/home/yike/phd/cancer_cells_img_seq/figures/batch_202112/segmentation/background_subtraction/bkg_sub_features.pkl'
with open(feas_fn1, 'rb') as f:
    features1 = pd.read_pickle(f)
features1.set_index('image', inplace=True)

# change index name
batch1 = sc.read_h5ad('/home/yike/phd/cancer_cells_img_seq/data/20220201_NextSeq/exon_filter.h5ad')
batch1.obs = batch1.obs[['R1', 'R2', 'batch', 'grid', '#cells', '#feature']]
batch1.obs.columns = ['R1', 'R2', 'Batch', 'Grid', 'Cell_number', 'Cell_annotation']
batch1.obs = batch1.obs.loc[features1.index]
features1.index = [i + '_' + j for (i, j) in zip(batch1.obs['Batch'].tolist(), batch1.obs_names.tolist())]

In [8]:
feas_fn2 = '/home/yike/phd/cancer_cells_img_seq/figures/batch_202306/background_substraction/bkg_sub_features.pkl'
with open(feas_fn2, 'rb') as f:
    features2 = pd.read_pickle(f)
features2.set_index('image', inplace=True)

# change index name
batch2 = sc.read_h5ad('/home/yike/phd/cancer_cells_img_seq/data/202306_NextSeq/202307_exon.h5ad')
batch2.obs['Batch'] = '3-1'
batch2.obs_names = [i + '_' + j for (i, j) in zip(batch2.obs['Batch'].tolist(), batch2.obs_names.tolist())]
batch2.obs['batch_index'] = np.array(batch2.obs_names.str.split('_').tolist())[:, 0]
features2.index = batch2.obs.reset_index().set_index('batch_index').loc[features2.index]['index']

In [56]:
# combine and save
features = pd.concat([features1, features2])

with open('/home/yike/phd/cancer_cells_img_seq/figures/combine_features.pkl', 'wb') as f:
    pickle.dump(features, f)